In [1]:
!pip install pydub
!pip install pandas
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 389.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 231.3 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 195.9 kB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 1.2 MB/s eta 0:00:00a 0:00:01m


In [2]:
!pip install midi2audio

In [3]:
!pip install pretty_midi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 397.4 kB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 547.5 kB/s eta 0:00:00a 0:00:01
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=139a45c88d17587e1c190dc27da3e8e2fab480e7b53087f53716afce4c0544e4
  Stored in directory: /home/cosiska/.cache/pip/wheels/e6/95/ac/15ceaeb2823b04d8e638fd1495357adb8d26c00ccac9d7782e
Successfully built pretty_midi


In [4]:
import time
import numpy
from midi2audio import FluidSynth
import pretty_midi
import pandas as pd
from pydub import AudioSegment
import subprocess
import mido

In [5]:
def reverse_midi(name, shift):
    midi_data = pretty_midi.PrettyMIDI(name + '.mid')
    notes_by_instruments = dict()
    instrument_number = 0
    instrument_types = dict()
    min = 127
    max = 0
    for instrument in midi_data.instruments:
        instrument_types[instrument_number] = instrument.program
        partiture = []
        for note in instrument.notes:
            partiture.append([note.pitch, note.start, note.end, note.velocity])
            if max < note.pitch:
                max = note.pitch
            if min > note.pitch:
                min = note.pitch 
        notes_by_instruments[instrument_number] = partiture
        instrument_number += 1
    
    reversed_midi = pretty_midi.PrettyMIDI()
    for instrument_number in instrument_types:
        partiture = pretty_midi.Instrument(program=instrument_types[instrument_number])
        for row in notes_by_instruments[instrument_number]:
            note = pretty_midi.Note(
                velocity = int(row[3]),
                pitch = int(max + min + shift - row[0]),
                start = float(row[1]),
                end = float(row[2]))
            partiture.notes.append(note)
        reversed_midi.instruments.append(partiture)
        
    reversed_midi.write(name + 'Reversed.mid')

    
# here input name of midi file and shift in half-tones
song_name = 'ThemeB'
reverse_midi(song_name, 16)
print('done!')

done!


In [7]:
# Conversion
fs = FluidSynth('weedsgm3.sf2')
fs.midi_to_audio('InTheEnd.mid', 'InTheEnd.wav')

fluidsynth: warning: Instrument 'Honkytonk Piano': Some invalid generators were discarded


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'InTheEnd.wav'..


In [8]:
AudioSegment.from_wav('InTheEnd.wav').export('InTheEnd.mp3', format="mp3")
print("Conversion finished!")

Conversion finished!
